In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy


tr_x = np.array([0] + [1.75 * np.sin(2*np.pi/4*i) for i in np.arange(4)] + [3.5 * np.sin(2*np.pi/12*i) for i in np.arange(12)])
tr_y = np.array([0] + [-1.75 * np.cos(2*np.pi/4*i) for i in np.arange(4)] + [-3.5 * np.cos(2*np.pi/12*i) for i in np.arange(12)])

fp_x = np.array([-3, 0, 3, -3, 0, 3])
fp_x_offset = np.array([-4, -1, 2, -4, -1, 2])
fp_y = np.array([7.7, 7.7, 7.7, -7.7, -7.7, -7.7])

fp_x_4 = np.array([0, 3, 0, 3])
fp_y_4 = np.array([7.7, 7.7, -7.7, -7.7])

fp_x_4_st41 = np.array([3, 0, -3, 0])
fp_x_4_st37_st39 = np.array([0, 3, -3, 0])

def multipole(N,s,A,x,y):
    r = np.sqrt(x**2+y**2)
    theta = np.arctan2(y,x)
    if(s==0):
        By = 61.79e6 + A * (r/4.5)**N * np.cos(N*theta)
        Bx = A * (r/4.5)**N * np.sin(N*theta)
    if(s==1):
        By = 61.79e6 + A * (r/4.5)**N * np.sin(N*theta)
        Bx = -A * (r/4.5)**N * np.cos(N*theta)
    B = np.sqrt(Bx**2 + By**2) - 61.79e6
    return B

Ns = [0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8];
ss = [0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1];

# THETA_TR = np.linalg.pinv(np.transpose(np.array([multipole(Ns[i],ss[i],1,tr_x,tr_y) for i in np.arange(17)])))

THETA_FP_6 = np.array([np.array([1, 1, 1, 1, 1, 1])/6.,  # dipole
                       np.array([1, 0, -1, 1, 0, -1])/-12.*4.5,  # n quad
                       np.array([1, 1, 1, -1, -1, -1])/46.2*4.5,  # s quad
                       np.array([1, 0, -1, -1, 0, 1])/-92.4/2.*4.5**2,  # s sext
                       np.array([1, -2, 1, 1, -2, 1])/18./2.*4.5**2,  # n sext
                       np.array([1, -2, 1, -1, 2, -1])/-138.6*4.5**3]  # NOT oct
                     )

THETA_FP_4 = np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2/2*4.5**2]  # sext
                     )

THETA_FP_4_ST37_ST39 = np.array([np.array([1, 0, 0, 1])/2.,  #dipole
                                 np.array([-1, 1, -1, 1])/6.*4.5,  # n quad
                                 np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                                 np.array([-1, 1, 1, -1])/46.2/2*4.5**2]  # sext?
                               )

THETA_FP_4_ST41 = np.array([np.array([0, 1, 0, 1])/2.,  #dipole
                            np.array([1, -1, -1, 1])/6.*4.5,  # n quad
                            np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                            np.array([1, -1, 1, -1])/46.2/2*4.5**2]  # sext?
                          )
                           

## Calculate Jacobian matrices

### Higher order fields into 6 probe stations

In [6]:
# n sext
B_fp = multipole(2, 0, 1, fp_x, fp_y)

print np.matmul(THETA_FP_6, B_fp)

#n dec
B_fp = multipole(4, 0, 1, fp_x, fp_y)

print np.matmul(THETA_FP_6, B_fp)

[-2.63160491  0.          0.          0.          1.00000009  0.        ]
[ 3.49913583e+00  0.00000000e+00 -1.11022302e-16  0.00000000e+00
 -1.71229606e+01 -1.77635684e-15]


In [7]:
# n sext
B_fp = multipole(2, 0, 1, fp_x_4, fp_y_4)

print np.matmul(THETA_FP_4, B_fp)

#n dec
B_fp = multipole(4, 0, 1, fp_x_4, fp_y_4)

print np.matmul(THETA_FP_4, B_fp)

[-2.92790124  0.66666673  0.          0.        ]
[  8.57260564 -11.41530708   0.           0.        ]


### 6-probe stations
Normal and (inflector chamber) offset

In [2]:
Nss = [[0,0], [1,0], [1,1], [2,1], [2,0]]

#####
J_6_PROBE = np.empty((5,5))
for i in range(5):
  Ns = Nss[i]

  B_fp = multipole(Ns[0], Ns[1], 1, fp_x, fp_y)
  J_6_PROBE[:, i] = np.matmul(THETA_FP_6, B_fp)[0:5]
J_6_PROBE = np.linalg.inv(J_6_PROBE)
J_6_PROBE[np.abs(J_6_PROBE)<1.0e-6] = 0

#####
J_6_PROBE_OFFSET = np.empty((5,5))
for i in range(5):
  Ns = Nss[i]

  B_fp = multipole(Ns[0], Ns[1], 1, fp_x_offset, fp_y)
  J_6_PROBE_OFFSET[:, i] = np.matmul(THETA_FP_6, B_fp)[0:5]
J_6_PROBE_OFFSET = np.linalg.inv(J_6_PROBE_OFFSET)
J_6_PROBE_OFFSET[np.abs(J_6_PROBE_OFFSET)<1.0e-6] = 0

#####
J_4_PROBE = np.zeros((5,5))
for i in range(5):
  Ns = Nss[i]

  B_fp = multipole(Ns[0], Ns[1], 1, fp_x_4, fp_y_4)
  J_4_PROBE[0:4, i] = np.matmul(THETA_FP_4, B_fp)[0:4]
J_4_PROBE[4,4] = 1
J_4_PROBE = np.linalg.inv(J_4_PROBE)
J_4_PROBE[np.abs(J_4_PROBE)<1.0e-6] = 0

#####
J_4_PROBE_ST41 = np.zeros((5,5))
for i in range(5):
  Ns = Nss[i]
  B_fp = multipole(Ns[0], Ns[1], 1, fp_x_4_st41, fp_y_4)
  J_4_PROBE_ST41[0:4, i] = np.matmul(THETA_FP_4_ST41, B_fp)[0:4]
J_4_PROBE_ST41[4,4] = 1
J_4_PROBE_ST41 = np.linalg.inv(J_4_PROBE_ST41)
J_4_PROBE_ST41[4,:] = np.array([0, 0, 0, 0, 1])
J_4_PROBE_ST41[np.abs(J_4_PROBE_ST41)<1.0e-6] = 0

#####
J_4_PROBE_ST37_ST39 = np.zeros((5,5))
for i in range(5):
  Ns = Nss[i]
  B_fp = multipole(Ns[0], Ns[1], 1, fp_x_4_st37_st39, fp_y_4)
  J_4_PROBE_ST37_ST39[0:4, i] = np.matmul(THETA_FP_4_ST37_ST39, B_fp)[0:4]
J_4_PROBE_ST37_ST39[4,4] = 1
J_4_PROBE_ST37_ST39 = np.linalg.inv(J_4_PROBE_ST37_ST39)
J_4_PROBE_ST37_ST39[4,:] = np.array([0, 0, 0, 0, 1])
J_4_PROBE_ST37_ST39[np.abs(J_4_PROBE_ST37_ST39)<1.0e-6] = 0

### Copy-Pastable Jacobians

In [3]:
print('J_6_PROBE = '+str([list(i) for i in np.round(J_6_PROBE,6)]))

print('J_6_PROBE_OFFSET = '+str([list(i) for i in np.round(J_6_PROBE_OFFSET,6)]))

print('J_4_PROBE = '+str([list(i) for i in np.round(J_4_PROBE,6)]))

print('J_4_PROBE_ST41 = '+str([list(i) for i in np.round(J_4_PROBE_ST41,6)]))
print('J_4_PROBE_ST37_ST39 = '+str([list(i) for i in np.round(J_4_PROBE_ST37_ST39,6)]))

J_6_PROBE = [[1.0, 0.0, 0.0, 0.0, 2.631605], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0]]
J_6_PROBE_OFFSET = [[1.0, 0.222222, 0.0, 0.0, 2.680987], [0.0, 1.0, 0.0, 0.0, 0.444444], [0.0, 0.0, 1.0, 0.444444, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0]]
J_4_PROBE = [[1.0, 0.0, 0.0, 0.0, 2.927901], [0.0, 1.0, 0.0, 0.0, -0.666667], [0.0, 0.0, 1.0, -0.666667, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0]]
J_4_PROBE_ST41 = [[1.0, 0.0, 0.0, 0.0, 2.927901], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, -0.194805, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, -0.194805], [0.0, 0.0, 0.0, 0.0, 1.0]]
J_4_PROBE_ST37_ST39 = [[1.0, 0.0, 0.0, 0.0, 2.927901], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, -0.194805, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, -0.194805], [0.0, 0.0, 0.0, 0.0, 1.0]]
